# Tarea 1 - Funnys Company 

## Grupo: La IA India de Hevia

- Nicolás Armijo
- Benjamín Camus
- Francisco Gonzalez
- Giovanni Mealla

In [23]:
import pulp as pl

# ----- datos ---- #
P_TIPO = (1, 2)                              # 1=Pequeña, 2=Grande # tipo de planta
CITIES = (1, 2, 3, 4, 5, 6)                  # ciudades
YEARS = (1, 2, 3)                           # años
REGIONS = (1, 2, 3, 4, 5, 6)                  # regiones
T_MODES = (1, 2, 3)                           # modos transporte AT1..AT3
CIUDAD = {1:"Antofagasta",2:"Valparaiso",3:"Santiago",4:"Rancagua",5:"Concepcion",6:"Puerto Montt"}

# Tabla 1 (demanda base y crecimiento)
D0 = {1: 951_776, 2: 967_364, 3: 512_051, 4: 386_248, 5: 946_174, 6: 303_445} # año 0, base
porcentaje = {1: 0.16,     2: 0.22,    3: 0.26,    4: 0.15,    5: 0.39,    6: 0.30   }

# Demanda por región y año (crecimiento compuesto)
D = {} #demanda de la region r en el año a.
for r in REGIONS: 
    for a in YEARS:
        D[(r,a)] = D0[r] * pow(1.0 + porcentaje[r], a)  # a=1..3
# es (region,año)
# D = {(1,1): 951_776, (1,2): , (1,3): 130, (2,1): , (2,2): , (2,3): , ...}


# Tabla 2 (capacidades anuales), Cp: capacidad de la planta tipo p
CAP = {1: 4_636_446, 2: 14_966_773}

PEQ, GRA = 1, 2

F = {
  (PEQ,1): 18_236_639, (PEQ,2):  8_838_286, (PEQ,3):  6_840_758,
  (PEQ,4): 13_378_246, (PEQ,5): 26_394_217, (PEQ,6):  3_678_737,
  (GRA,1): 60_788_796, (GRA,2): 32_734_393, (GRA,3): 32_575_039,
  (GRA,4): 53_512_984, (GRA,5): 65_985_543, (GRA,6): 26_276_695,
}

#costo de produccir una unidad en una planta p en una ciudad r
V = {
  (PEQ,1): 28.20, (PEQ,2): 41.68, (PEQ,3): 18.57, (PEQ,4): 17.68, (PEQ,5): 50.11, (PEQ,6): 43.55,
  (GRA,1): 28.20, (GRA,2): 41.68, (GRA,3): 18.57, (GRA,4): 17.68, (GRA,5): 50.11, (GRA,6): 43.55,
}

#costo de apertura de planta tipo p en en la ciudad c
A = {
  (PEQ,1):  86_626_147, (PEQ,2): 115_721_215, (PEQ,3): 172_235_977,
  (PEQ,4):           0, (PEQ,5):  57_494_934, (PEQ,6): 175_561_277,
  (GRA,1): 201_456_157, (GRA,2): 199_519_337, (GRA,3): 291_925_385,
  (GRA,4): 299_031_830, (GRA,5): 179_671_671, (GRA,6): 337_617_842,
}

# Tablas 4-6 (costos de transporte T[c,r,t])
# T1
AT1 = {
  1:[1.06, 2.80,10.29, 4.87, 6.41,10.35],
  2:[3.49, 6.19, 3.39, 6.77, 3.07, 6.61],
  3:[6.38, 5.88, 5.63, 1.01, 3.15, 5.67],
  4:[3.44, 1.48, 2.79, 2.80, 5.30, 1.29],
  5:[5.94, 7.33, 1.80, 9.48, 2.82, 8.25],
  6:[2.57, 9.63, 4.84, 6.64, 6.48, 8.54],
}
# AT2
AT2 = {
  1:[10.03, 4.09, 4.55, 7.84, 5.33,10.63],
  2:[10.52, 1.82, 3.91, 8.20, 5.88, 2.33],
  3:[ 1.90, 8.89, 6.55, 9.71, 7.03,10.23],
  4:[ 2.06,10.17, 2.12, 6.11, 3.79, 6.19],
  5:[ 2.54, 6.95, 8.57,10.50, 4.85, 5.31],
  6:[ 7.92,10.32, 1.41, 4.94, 2.74, 8.08],
}
# AT3
AT3 = {
  1:[ 9.86, 4.30, 8.10, 9.63, 7.40, 6.47],
  2:[ 1.58, 2.71, 3.08, 5.91, 7.99, 5.11],
  3:[ 9.13,10.03, 6.77, 5.70, 3.62, 8.58],
  4:[ 8.95, 7.37, 10.29, 3.34, 2.21, 4.58],
  5:[ 9.62, 3.78, 5.19, 2.61, 3.19, 1.78],
  6:[10.32, 8.88, 10.87,10.38, 5.83, 1.54],
}

# construir matriz de costos de transporte T[c,r,t], costo de transporte de una unidad, desde la ciudad c a la region r en el año t
T = {}
for c in CITIES:
    for r in REGIONS:
        T[(c,r,1)] = AT1[c][r-1] # transporte AT1
        T[(c,r,2)] = AT2[c][r-1] # transporte AT2
        T[(c,r,3)] = AT3[c][r-1] # transporte AT3

# MODELO
prob = pl.LpProblem("Funnys", pl.LpMinimize)

# Variables
x   = pl.LpVariable.dicts("x",   [(p,c) for p in P_TIPO for c in CITIES], 0, 1, cat=pl.LpBinary)
y = pl.LpVariable.dicts(
    "y", [(p,c,r,t,a) for p in P_TIPO for c in CITIES for r in REGIONS for t in T_MODES for a in YEARS ],
    lowBound=0, cat=pl.LpInteger
)

# Función objetivo
prob += (
    pl.lpSum(((3*F[(p,c)] + A[(p,c)]) * x[(p,c)]) for p in P_TIPO for c in CITIES)
  + pl.lpSum( V[(p,c)]    * y[(p,c,r,t,a)]        for p in P_TIPO for c in CITIES for r in REGIONS for t in T_MODES for a in YEARS)
  + pl.lpSum( T[(c,r,t)]  * y[(p,c,r,t,a)]        for p in P_TIPO for c in CITIES for r in REGIONS for t in T_MODES for a in YEARS)
), "z"

#esta es una version con la sumatoria factorizada, que a la final es lo mismo
#prob += (pl.lpSum(((3*F[p,c] + A[p,c])*x[p,c] for p in P_TIPO for c in CITIES)) 
#         +pl.lpSum(V[(p,c)] * y[(p,c,r,t,a)] for p in P_TIPO for c in CITIES for r in REGIONS for t in T_MODES for a in YEARS )
#         + pl.lpSum(T[c,r,t]* pl.lpSum(y[p,c,r,t,a] for a in YEARS) for p in P_TIPO for c in CITIES for r in REGIONS for t in T_MODES)), "z"


# RESTRICCIONES

#(1) Demanda exacta por región y año
for r in REGIONS:
    for a in YEARS:
        prob += pl.lpSum(y[(p,c,r,t,a)] for p in P_TIPO for c in CITIES for t in T_MODES) >= D[(r,a)], f"Dem_{r}_{a}"



# (2) Capacidad por ciudad–tipo–año
for p in P_TIPO:
    for c in CITIES:
        for a in YEARS:
            prob += pl.lpSum(y[(p,c,r,t,a)] for r in REGIONS for t in T_MODES) <= CAP[p]*x[(p,c)], f"Cap_{p}_{c}_{a}"

#  (3)
for c in CITIES:
  prob += pl.lpSum(x[p,c] for p in P_TIPO) <= 1, f"planta_unica_{c}"



# RESOLVER (CBC por defecto en PuLP)

solver = pl.PULP_CBC_CMD(msg=True)
prob.solve(solver)

print("Estado:", pl.LpStatus[prob.status])
print("Valor objetivo z =", pl.value(prob.objective))


for c in CITIES:
    opened = [p for p in P_TIPO if x[p,c].varValue and x[p,c].varValue > 0]
    for p in opened:
        print(f"  Ciudad {c} ({CIUDAD[c]}): Planta {'Peq' if p==1 else 'Gra'} abierta")




Estado: Optimal
Valor objetivo z = 619887317.12
  Ciudad 3 (Santiago): Planta Peq abierta
  Ciudad 4 (Rancagua): Planta Peq abierta


## Validación

In [ ]:
# AUDITORIA
outfile = "auditoria_resultados.txt"

def _v(var):
    val = getattr(var, "varValue", None)
    return 0.0 if val is None else float(val)

mode_name = {1: "AT1", 2: "AT2", 3: "AT3"}

with open(outfile, "w") as f:
    f.write("=== Auditoría de Resultados del Modelo ===\n")
    f.write(f"Estado del modelo: {pl.LpStatus[prob.status]}\n")
    f.write(f"Valor objetivo (costo total): {pl.value(prob.objective):,.2f}\n\n")

    # 1) Plantas abiertas
    f.write("Plantas abiertas:\n")
    for c in CITIES:
        for p in P_TIPO:
            if _v(x[(p, c)]) > 0.5:
                f.write(f"  Ciudad {c} ({CIUDAD[c]}): {'Pequeña' if p==1 else 'Grande'}\n")
    f.write("\n")

    # 2) Costos desglosados y reconciliación con FO
    cost_open_fix = sum((3*F[(p,c)] + A[(p,c)]) * _v(x[(p,c)]) for p in P_TIPO for c in CITIES)
    cost_var      = sum(V[(p,c)] * _v(y[(p,c,r,t,a)]) for p in P_TIPO for c in CITIES
                                                     for r in REGIONS for t in T_MODES for a in YEARS)
    cost_ship     = sum(T[(c,r,t)] * _v(y[(p,c,r,t,a)]) for p in P_TIPO for c in CITIES
                                                        for r in REGIONS for t in T_MODES for a in YEARS)
    f.write("Costos:\n")
    f.write(f"  Apertura+Fijos (3 años): {cost_open_fix:,.2f}\n")
    f.write(f"  Producción:              {cost_var:,.2f}\n")
    f.write(f"  Transporte:              {cost_ship:,.2f}\n")
    f.write(f"  Total (recalc):          {cost_open_fix + cost_var + cost_ship:,.2f}\n")
    f.write(f"  Total (objetivo):        {pl.value(prob.objective):,.2f}\n\n")

    # 3) Verificación de demanda y gap
    f.write("Satisfacción de Demanda (Región, Año):\n")
    for r in REGIONS:
        for a in YEARS:
            shipped = sum(_v(y[(p,c,r,t,a)]) for p in P_TIPO for c in CITIES for t in T_MODES)
            dem = D[(r, a)]
            gap = shipped - dem
            f.write(f"  R{r}, Año {a}: Enviado={shipped:,.0f}  Demanda={dem:,.0f}  Gap={gap:,.4f}\n")
    f.write("\n")

    # 4) Utilización de capacidad por ciudad-tipo-año
    f.write("Utilización de Capacidad (Ciudad, Tipo, Año):\n")
    for c in CITIES:
        for p in P_TIPO:
            for a in YEARS:
                prod = sum(_v(y[(p,c,r,t,a)]) for r in REGIONS for t in T_MODES)
                cap  = CAP[p] * _v(x[(p,c)])
                util = (prod / cap * 100.0) if cap > 0 else 0.0
                if cap > 0 or prod > 1e-6:
                    f.write(f"  {CIUDAD[c]:<12} | {'Peq' if p==1 else 'Gra'} | Año {a}: "
                            f"Prod={prod:,.0f}  Cap={cap:,.0f}  Util={util:5.1f}%\n")
    f.write("\n")

    # 5) Envíos positivos
    f.write("Envíos positivos (origen→región | modo | año | tipo):\n")
    for c in CITIES:
        for r in REGIONS:
            for t in T_MODES:
                for a in YEARS:
                    for p in P_TIPO:
                        q = _v(y[(p,c,r,t,a)])
                        if q > 1e-6:
                            f.write(f"  {CIUDAD[c]:<12} → R{r} | {mode_name[t]} | Año {a} | "
                                    f"{'Peq' if p==1 else 'Gra'}: {q:,.0f}\n")
    f.write("\n")

    # 6) Chequeo de sanidad (no producir sin planta)
    f.write("Chequeo: producción solo donde hay planta:\n")
    ok = True
    for c in CITIES:
        for p in P_TIPO:
            if _v(x[(p,c)]) < 0.5:
                prod_tot = sum(_v(y[(p,c,r,t,a)]) for r in REGIONS for t in T_MODES for a in YEARS)
                if prod_tot > 1e-6:
                    ok = False
                    f.write(f"  !!ERROR: {CIUDAD[c]} {('Peq' if p==1 else 'Gra')} produce {prod_tot:,.0f} sin planta.\n")
    if ok:
        f.write("  OK: no hay producción en tipos/ciudades sin planta instalada.\n")

print(f"Archivo de auditoría generado: {outfile}")


Archivo de auditoría generado: auditoria_resultados.txt
